In [29]:
!pip install python-docx

In [37]:
import requests
from docx import Document
import os

subscription_key = "***"
endpoint = 'https://api.cognitive.microsofttranslator.com/'
location = "eastus"
language_destination = "pt-br"

def translator_text(text, target_language):
  path = '/translate'
  constructed_url = endpoint + path
  headers = {
      'Ocp-Apim-Subscription-Key': subscription_key,
      'Ocp-Apim-Subscription-Region': location,
      'Content-type': 'application/json',
      'X-ClientTraceId': str(os.urandom(16))
  }

  body = [{
      'text': text
  }]
  params = {
    'api-version': '3.0',
    'from': 'en',
    'to': [target_language]
}
  response = requests.post(constructed_url, headers=headers, json=body, params=params).json()
  return response[0]["translations"][0]["text"]

In [38]:
translator_text("I know you're somewhere out there, Somewhere far away", language_destination)

'Eu sei que você está em algum lugar lá fora, em algum lugar distante'

In [39]:
def translate_document(path):
    document = Document(path)
    full_text = []
    for paragraph in document.paragraphs:
      translated_text = translator_text(paragraph.text, language_destination)
      full_text.append(translated_text)

    translated_doc = Document()
    for line in full_text:
      print(line)
      translated_doc.add_paragraph(line)
      path_translated = path.replace(".docx", f"_{language_destination}.docx")
      translated_doc.save(path_translated)
      return path_translated

input_file = "/content/MUSICA_BRUNO_MARS.docx"
translate_document(input_file)

Eu sei que você está em algum lugar lá fora
Em algum lugar distante
Eu quero você de volta
Eu quero você de volta


'/content/MUSICA_BRUNO_MARS_pt-br.docx'

In [69]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()
      texto = soup.get_text(separator= '')
      #limpar texto
      linhas = (line.strip() for line in texto.splitlines())
      parts = (phrase.strip() for line in linhas for phrase in line.split("  "))
      texto_limpo = '\n'.join(part for part in parts if part)
      return texto_limpo
    else:
      print(f"Falha ao acessar a URL. Código de status: {response.status_code}")
      return None

    text = soup.get_text()
    return text

extract_text_from_url('https://dev.to/bascodes/this-week-in-python-4bh9')


"This Week In Python - DEV Community\nSkip to content\nNavigation menu\nSearch\nPowered by\nSearch\nAlgolia\nSearch\nLog in\nCreate account\nDEV Community\nClose\nAdd reaction\nLike\nUnicorn\nExploding Head\nRaised Hands\nFire\nJump to Comments\nSave\nMore...\nCopy link\nCopy link\nCopied to Clipboard\nShare to X\nShare to LinkedIn\nShare to Facebook\nShare to Mastodon\nReport Abuse\nBas Steins\nPosted on Nov 15\n• Originally published at bas.codes\nThis Week In Python\n#python\n#thisweekinpython\nFri, November 15, 2024\nThis Week in Python is a concise reading list about what happened in the past week in the Python universe.\nPython Articles\nThoughts on Django’s Core\nAttestations: A new generation of signatures on PyPI\nThe Pythonic Emptiness\nResearchers develop Python code for in-memory computing\nML in Go with a Python sidecar\nProjects\nTinyTroupe – LLM-powered multiagent persona simulation for imagination enhancement and business insights\ndenormalized – Embeddable stream proce

In [70]:
#!pip install -U langchain-openai
from langchain_openai.chat_models.azure import AzureChatOpenAI

client = AzureChatOpenAI(
    azure_endpoint= "https://oai-dio-bootcamp-dev-eastus-1.openai.azure.com/",
    api_key= "***",
    api_version= "2024-02-15-preview",
    deployment_name= "gpt-4o-mini",
    max_retries=0
)

def translate_article(text, lang):
  messages = [
      ("system" , "Você atua como tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
  ]

  response = client.invoke(messages)
  print(response.content)
  return response.content

translate_article("This Week in Python is a concise reading list about what happened in the past week in the Python universe.", "portugues")

```markdown
Esta Semana em Python é uma lista de leitura concisa sobre o que aconteceu na última semana no universo Python.
```


'```markdown\nEsta Semana em Python é uma lista de leitura concisa sobre o que aconteceu na última semana no universo Python.\n```'

In [71]:
url = "https://dev.to/bascodes/this-week-in-python-4bh9"
text = extract_text_from_url(url)
translate_article(text, "portugues")

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-15-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}